### Getting Started

In [1]:
# importing libraries
import urllib3
import json
from bs4 import BeautifulSoup

In [2]:
http = urllib3.PoolManager()

In [66]:
url = "https://www.airlinequality.com/airline-reviews/air-astana"
response = http.request('GET', url)
print(response.status)
soup = BeautifulSoup(response.data)

200


### Getting every page

In [10]:
url

'https://www.airlinequality.com/airline-reviews/air-astana'

In [54]:
urls = []
for i in range(25):
    pages = f'/page/{i}/'
    urls.append(url+pages)

urls[15]

'https://www.airlinequality.com/airline-reviews/air-astana/page/15/'

### Getting each review

In [74]:
def get_info(url, count):
    response = http.request('GET', url)
    if response.status != 200:
        d = {"error": response.status}
        return d
    soup = BeautifulSoup(response.data)
    
    topic = soup.find_all("h2", {"text_header"})
    user = soup.find_all("h3", {"text_sub_header userStatusWrapper"})
    review = soup.find_all("div", {"text_content"})
    ratings = soup.find_all("div", "review-stats")

    users = []
    for i in range(len(topic)):
        reviews = {}
        reviews['id'] = i+count
        reviews['Topic'] = topic[i].get_text()
        try:
            try:
                number = user[i].span.find_all("span", "userStatusReviewCount")[0].text
                name = user[i].text
                name = name.replace(user[i].time.text, "")
                date = user[i].text.replace(name, "")
                name = name.replace('\n\n'+number+'\n\n\n\n', "")

            except:
                name = user[i].text
                name = name.replace(user[i].time.text, "")
                date = user[i].text.replace(name, "")
                name = name.replace('\n\n', "")
        except:
            continue

            
        reviews['Name'] = name
        reviews['Date'] = date
        
        try:
            verified = review[i].find("a").get_text()
            text = review[i].get_text().replace(verified + " |", "")
        except:
            verified = None
            text = review[i].get_text()
        reviews['Verified'] = verified
        reviews['Text'] = text
        
        # getting ratings
        rows = ratings[i].find_all("tr")
        for j in rows:
            name = j.find_all(("td", "review-value"))[0].get_text()
            value = j.find_all(("td", "review-value"))[1].get_text()
            if value == '12345':
                value = len(j.find_all("span", "star fill"))
            reviews[name] = value
                
        users.append(reviews)

    return users, i+count

data, count = get_info(url, 0)
data

[{'id': 0,
  'Topic': '"rescheduled for a morning flight"',
  'Name': 'M Badeil (Australia) ',
  'Date': '11th May 2023',
  'Verified': 'Trip Verified',
  'Text': '✅   I booked with Air Astana for a flight from Tashkent to Astana transiting in Almaty. Our flight was delayed for over an hour in Astana - an additional one hour later than my original booking. This meant that I now have too short a transit time and need to be re-scheduled. I received an SMS advising that I am rescheduled for a morning flight (arrival 10 a.m. the next day instead of 6 pm) I then went into frantic messages to my hotel in Astana cancelling my booking - about AUD100 extra for a few hours! On check-in I was moved to another later flight with no discussion with me. So I am unaware of this, land in Almaty transit and go to the counter to check my hotel voucher. There, I am directed to the amended boarding pass and it is for a night flight arriving 10pm. So, again, frantic messages to my hotel - after over 20 minu

In [76]:
url

'https://www.airlinequality.com/airline-reviews/air-astana'

### Scraping every page 

In [77]:
data = []
count = 0
for url in urls:
#     print(i)
    d, count = get_info(url, count)
    data.extend(d)
    count+=1

### DataFrame

In [80]:
import pandas as pd
df = pd.DataFrame(data)
df

,id,Topic,Name,Date,Verified,Text,Type Of Traveller,Seat Type,Route,Date Flown,Seat Comfort,Cabin Staff Service,Food & Beverages,Inflight Entertainment,Ground Service,Wifi & Connectivity,Value For Money,Recommended,Aircraft
0,0,"""rescheduled for a morning flight""",M Badeil (Australia),11th May 2023,Trip Verified,✅ I booked with Air Astana for a flight from...,Solo Leisure,Economy Class,Tashkent to Astana via Almaty,May 2023,3,3,3,3,2.0,2.0,3,no,NaN
1,1,"""I haven't got any repayment""",Natalia Onishchenko (Kyrgyzstan),10th March 2023,Trip Verified,✅ The flight that I report was supposed to t...,Family Leisure,Economy Class,Moscow to Nur-Sultan via Almaty / Bishkek,October 2022,NaN,NaN,NaN,NaN,1.0,NaN,2,no,NaN
2,2,"""This is how an airline should be run!""",Adrian Pritchard (United Kingdom),26th February 2023,Trip Verified,✅ What a nice little airline! Air Astana prov...,Solo Leisure,Economy Class,Istanbul to Astana,January 2023,5,5,5,5,5.0,5.0,5,yes,NaN
3,3,"""an amazing experience""",Ruby Phul (Turkey),24th November 2022,Not Verified,It was an amazing experience on all the 4 fl...,Solo Leisure,Economy Class,Istanbul to Delhi via Almaty,April 2022,5,5,5,4,4.0,4.0,5,yes,NaN
4,4,"""No lounge or priority check in""",E Kalesova (Kazakhstan),4th October 2022,Trip Verified,"✅ Everything was perfect, but not in Atyrau ...",Solo Leisure,Business Class,Atyrau to Almaty,September 2022,5,5,5,5,2.0,NaN,3,no,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,340,Air Astana customer review,S Bryant (United Kingdom),7th July 2009,None,The comments about this airline mostly relate ...,NaN,Economy Class,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,no,NaN
245,341,Air Astana customer review,I Williamson (Saudi Arabia),21st June 2009,None,Air Astana is right up there with other non-ma...,NaN,Economy Class,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,yes,NaN
246,342,Air Astana customer review,J Williams (United Kingdom),4th June 2009,None,Almaty - Atyrau - Amsterdam in Business. Seats...,NaN,Business Class,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,yes,NaN
247,343,Air Astana customer review,S Ivanov (Kazakhstan),29th March 2009,None,The have good cabin crew quality of food is go...,NaN,Economy Class,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,yes,NaN


In [82]:
df.to_csv('AirAstanaReview.csv', index=False)